In [1]:
import pandas as pd
import numpy as np
import io
import math
import random
import matplotlib.pyplot as plt
import librosa
import librosa.display
import cv2
import cmapy
import nlpaug
import nlpaug.augmenter.audio as naa
from scipy.signal import butter, lfilter
import torch
from tqdm import tqdm
import os

In [2]:
audio_train=pd.read_csv('Temp_test_audio.csv')
audio_train.head(4)

,Unnamed: 0,Patient_ID,Disease,Recording_index,Chest_Location,Acquisition_Mode,Recording_equipment,Respiratory_cycles,Normal_cycles,Wheeze_cycles,Crackle_cycles,Both,Filename,Train/Test
0,217,136,NaN,1b1,Ar,sc,Meditron,9,9,0,0,0,136_1b1_Ar_sc_Meditron.txt,test
1,247,139,NaN,1b1,Al,sc,Litt3200,10,10,0,0,0,139_1b1_Al_sc_Litt3200.txt,test
2,248,139,NaN,1b1,Ar,sc,Litt3200,9,9,0,0,0,139_1b1_Ar_sc_Litt3200.txt,test
3,249,139,NaN,1b1,Ll,sc,Litt3200,8,7,0,0,1,139_1b1_Ll_sc_Litt3200.txt,test


In [3]:
sample_rate=4000
fs=30000
data_dir='AudioFiles2/'
desired_length=8
n_mels = 64
nfft = 256
hop = nfft//2
f_max = 2000
train_flag=1
lowcut=50
highcut=2000

In [4]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b,a, data)
    return y

In [5]:
#slicing into breath cycles
def slice_data(start, end, raw_data, sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    y=butter_bandpass_filter(raw_data[start_ind: end_ind], lowcut, highcut, fs, order=5)
    return y

In [6]:
#getting label of each cycles
def get_label(crackle, wheeze):
    if crackle == 0 and wheeze == 0:
        return 0
    elif crackle == 1 and wheeze == 0:
        return 1
    elif crackle == 0 and wheeze == 1:
        return 2
    else:
        return 3

In [7]:
#getting breath cycle samples
def get_sound_samples(file_name, data_dir, sample_rate):
    sample_data = [file_name]
    data, rate = librosa.load(os.path.join(data_dir, file_name+'.wav'), sr=sample_rate)
    recording_annotations=pd.read_csv(data_dir+file_name+'.txt', sep='\t', header=None)
    
    for i in range(len(recording_annotations.index)):
        row = recording_annotations.loc[i]
        start = row[0]
        end = row[1]
        crackles = row[2]
        wheezes = row[3]
        audio_chunk = slice_data(start, end, data, rate)
        sample_data.append((audio_chunk, start,end, get_label(crackles, wheezes)))
    return sample_data

In [8]:
#creating list of breath cycles with respective labels
filenames=audio_train['Filename']
filenames_with_labels=[]
cycle_list = []
classwise_cycle_list = []
for idx, filename in tqdm(enumerate(filenames)):
            file_name=filename.split('.')[0]
            print(file_name+" going on "+str(idx)+" out of "+str(len(filenames)))
            data = get_sound_samples(file_name, data_dir, sample_rate)
            cycles_with_labels = [(d[0], d[3], file_name, cycle_idx, 0) for cycle_idx, d in enumerate(data[1:])]
            cycle_list.extend(cycles_with_labels)
            for cycle_idx, d in enumerate(cycles_with_labels):
                filenames_with_labels.append(file_name+'_'+str(d[3])+'_'+str(d[1]))
                classwise_cycle_list.append((d[0],d[1]))

0it [00:00, ?it/s]

136_1b1_Ar_sc_Meditron going on 0 out of 10


5it [00:03,  2.06it/s]

139_1b1_Al_sc_Litt3200 going on 1 out of 10
139_1b1_Ar_sc_Litt3200 going on 2 out of 10
139_1b1_Ll_sc_Litt3200 going on 3 out of 10
139_1b1_Lr_sc_Litt3200 going on 4 out of 10
139_1b1_Pl_sc_Litt3200 going on 5 out of 10
139_1b1_Pr_sc_Litt3200 going on 6 out of 10
143_1b1_Al_sc_Meditron going on 7 out of 10


8it [00:04,  2.59it/s]

144_1b1_Al_sc_Meditron going on 8 out of 10
144_1b1_Tc_sc_Meditron going on 9 out of 10


10it [00:05,  1.78it/s]


In [9]:
cycle_list[0][3]

0

In [11]:
largest_stft=librosa.stft(cycle_list[id][0], n_fft=512, hop_length = 64)
largest_stft.shape

(257, 331)

In [12]:
def padding(array, xx, yy):
    """
    :param array: numpy array
    :param xx: desired height
    :param yy: desired width
    :return: padded array
    """
    h = array.shape[0]
    w = array.shape[1]
    a = max((xx - h) // 2,0)
    aa = max(0,xx - a - h)
    b = max(0,(yy - w) // 2)
    bb = max(yy - b - w,0)
    return np.pad(array, pad_width=((a, aa), (b, bb)), mode='constant')

In [13]:
## creating sliced and padded data = audio_data
stft_data=[]
for idx, sample in enumerate(cycle_list):
            print(str(idx)+" out of "+str(len(cycle_list)))
            stft=librosa.stft(sample[0], n_fft=512, hop_length = 64)
            stft_data.append(padding(stft, 257, 1011))

print(stft.shape, sample[0].shape)
print(stft_data[4].shape)

0 out of 93
1 out of 93
2 out of 93
3 out of 93
4 out of 93
5 out of 93
6 out of 93
7 out of 93
8 out of 93
9 out of 93
10 out of 93
11 out of 93
12 out of 93
13 out of 93
14 out of 93
15 out of 93
16 out of 93
17 out of 93
18 out of 93
19 out of 93
20 out of 93
21 out of 93
22 out of 93
23 out of 93
24 out of 93
25 out of 93
26 out of 93
27 out of 93
28 out of 93
29 out of 93
30 out of 93
31 out of 93
32 out of 93
33 out of 93
34 out of 93
35 out of 93
36 out of 93
37 out of 93
38 out of 93
39 out of 93
40 out of 93
41 out of 93
42 out of 93
43 out of 93
44 out of 93
45 out of 93
46 out of 93
47 out of 93
48 out of 93
49 out of 93
50 out of 93
51 out of 93
52 out of 93
53 out of 93
54 out of 93
55 out of 93
56 out of 93
57 out of 93
58 out of 93
59 out of 93
60 out of 93
61 out of 93
62 out of 93
63 out of 93
64 out of 93
65 out of 93
66 out of 93
67 out of 93
68 out of 93
69 out of 93
70 out of 93
71 out of 93
72 out of 93
73 out of 93
74 out of 93
75 out of 93
76 out of 93
77 out of

In [ ]:
stft_data[0]

In [ ]:
largest=0
file=""
cycle_id=0
id=0
for i in range(2756):
    if largest<stft_data[i][0].shape[0]:
        largest=stft_data[i][0].shape[0]
        file=cycle_list[i][2]
        cycle_id=cycle_list[i][3]
        id=i
    print(i)
print(largest, file, cycle_id, id)

In [2]:
train_dataset=[]
for index in range(len(stft_data)):
    stft=stft_data[index]
    label = classwise_cycle_list[index][1]
    train_dataset.append((stft, label))

NameError: name 'stft_data' is not defined

In [1]:
X_train=[]
y_train=[]
for i in range(len(train_dataset)):
    X_train.append(train_dataset[i][0])
    y_train.append(train_dataset[i][1])  
    
np.save('X_train_STFT_small', np.concatenate(X_train, axis=0).reshape(len(train_dataset), 257, 1011, 1))
#np.save('X_test_STFT_small', X_train)
np.save('y_train_STFT_small', y_train)

NameError: name 'train_dataset' is not defined